In [135]:
import configparser
import csv
from collections import Counter
import nbimporter
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import math

### Lendo arquivo de configuração

In [136]:
import ast

In [137]:
config = configparser.ConfigParser()
config.read('BUSCA.cfg')

modelo_path = config.get('DEFAULT', 'MODELO')

modelo_vetorial = {}
with open(modelo_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    
    for row in reader:
        key_str = row[0].replace("(", "").replace(")", "")
        key_tuple = tuple(ast.literal_eval(key_str))
        value = float(row[1])
        
        modelo_vetorial[key_tuple] = value

consultas_path = config.get('DEFAULT', 'CONSULTAS')
results_path = config.get('DEFAULT', 'RESULTADOS')

In [138]:
queries = {}

with open(consultas_path, "r") as file:
    for id_num, line in enumerate(file, 1):  # Start enumeration from 1
        # Split the line based on the comma separator
        consultas = line.strip().split(",")
        
        # Assign each string to the corresponding ID number in the dictionary
        for consulta in consultas:
            queries[id_num] = consulta
            id_num += 1



In [139]:
queries

{1: 'respiratory infection with cystic fibrosis complications',
 2: 'tract significance'}

In [140]:
modelo_vetorial

{('00001 ', 'SIGNIFICANCE'): 0.04154799294201786,
 ('00074 ', 'SIGNIFICANCE'): 0.08781734871835592,
 ('00078 ', 'SIGNIFICANCE'): 0.07576398712956198,
 ('00121 ', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00147 ', 'SIGNIFICANCE'): 0.04067329835376485,
 ('00157 ', 'SIGNIFICANCE'): 0.14310975346695037,
 ('00179', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00185', 'SIGNIFICANCE'): 0.028621950693390078,
 ('00195', 'SIGNIFICANCE'): 0.0536661575501064,
 ('00205', 'SIGNIFICANCE'): 0.03018721362193485,
 ('00223', 'SIGNIFICANCE'): 0.058544899145570616,
 ('00258', 'SIGNIFICANCE'): 0.019514966381856874,
 ('00319', 'SIGNIFICANCE'): 0.052215720859562983,
 ('00355', 'SIGNIFICANCE'): 0.04246113564404023,
 ('00402 ', 'SIGNIFICANCE'): 0.12879877812025536,
 ('00411 ', 'SIGNIFICANCE'): 0.031933581352129425,
 ('00412 ', 'SIGNIFICANCE'): 0.08221198603420554,
 ('00485 ', 'SIGNIFICANCE'): 0.0633436613706174,
 ('00526 ', 'SIGNIFICANCE'): 0.0330253277231424,
 ('00555 ', 'SIGNIFICANCE'): 0.06899934542156537,
 ('0

In [141]:
%store -r li
li

{'SIGNIFICANCE': ['00001 ',
  '00074 ',
  '00078 ',
  '00121 ',
  '00147 ',
  '00157 ',
  '00179',
  '00185',
  '00195',
  '00205',
  '00223',
  '00258',
  '00319',
  '00355',
  '00402 ',
  '00411 ',
  '00412 ',
  '00485 ',
  '00526 ',
  '00555 ',
  '00593 ',
  '00631 ',
  '00809 ',
  '00858 ',
  '00923 ',
  '01151 '],
 'PSEUDOMONAS': ['00001 ',
  '00001 ',
  '00001 ',
  '00007 ',
  '00008 ',
  '00018 ',
  '00018 ',
  '00061 ',
  '00061 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00079 ',
  '00080 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00082 ',
  '00082 ',
  '00082 ',
  '00123 ',
  '00123 ',
  '00123 ',
  '00135 ',
  '00148 ',
  '00152 ',
  '00159 ',
  '00160 ',
  '00161 ',
  '00176',
  '00177',
  '00177',
  '00178',
  '00188',
  '00188',
  '00188',
  '00188',
  '00200',
  '00200',
  '00250',
  '00260',
  '00265',
  '00271',
  '00282',
  '00282',
  '00384 ',
  '00384 ',
  '00394 ',
  '00427 ',
  '00427 ',
  '00447 ',
  '00458 ',
  '00459 ',
  '00460

In [142]:
%store -r n_words
%store -r li
N = len(n_words.keys())

In [143]:
li

{'SIGNIFICANCE': ['00001 ',
  '00074 ',
  '00078 ',
  '00121 ',
  '00147 ',
  '00157 ',
  '00179',
  '00185',
  '00195',
  '00205',
  '00223',
  '00258',
  '00319',
  '00355',
  '00402 ',
  '00411 ',
  '00412 ',
  '00485 ',
  '00526 ',
  '00555 ',
  '00593 ',
  '00631 ',
  '00809 ',
  '00858 ',
  '00923 ',
  '01151 '],
 'PSEUDOMONAS': ['00001 ',
  '00001 ',
  '00001 ',
  '00007 ',
  '00008 ',
  '00018 ',
  '00018 ',
  '00061 ',
  '00061 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00062 ',
  '00079 ',
  '00080 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00081 ',
  '00082 ',
  '00082 ',
  '00082 ',
  '00123 ',
  '00123 ',
  '00123 ',
  '00135 ',
  '00148 ',
  '00152 ',
  '00159 ',
  '00160 ',
  '00161 ',
  '00176',
  '00177',
  '00177',
  '00178',
  '00188',
  '00188',
  '00188',
  '00188',
  '00200',
  '00200',
  '00250',
  '00260',
  '00265',
  '00271',
  '00282',
  '00282',
  '00384 ',
  '00384 ',
  '00394 ',
  '00427 ',
  '00427 ',
  '00447 ',
  '00458 ',
  '00459 ',
  '00460

In [144]:
modelo_vetorial

{('00001 ', 'SIGNIFICANCE'): 0.04154799294201786,
 ('00074 ', 'SIGNIFICANCE'): 0.08781734871835592,
 ('00078 ', 'SIGNIFICANCE'): 0.07576398712956198,
 ('00121 ', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00147 ', 'SIGNIFICANCE'): 0.04067329835376485,
 ('00157 ', 'SIGNIFICANCE'): 0.14310975346695037,
 ('00179', 'SIGNIFICANCE'): 0.034194365872634164,
 ('00185', 'SIGNIFICANCE'): 0.028621950693390078,
 ('00195', 'SIGNIFICANCE'): 0.0536661575501064,
 ('00205', 'SIGNIFICANCE'): 0.03018721362193485,
 ('00223', 'SIGNIFICANCE'): 0.058544899145570616,
 ('00258', 'SIGNIFICANCE'): 0.019514966381856874,
 ('00319', 'SIGNIFICANCE'): 0.052215720859562983,
 ('00355', 'SIGNIFICANCE'): 0.04246113564404023,
 ('00402 ', 'SIGNIFICANCE'): 0.12879877812025536,
 ('00411 ', 'SIGNIFICANCE'): 0.031933581352129425,
 ('00412 ', 'SIGNIFICANCE'): 0.08221198603420554,
 ('00485 ', 'SIGNIFICANCE'): 0.0633436613706174,
 ('00526 ', 'SIGNIFICANCE'): 0.0330253277231424,
 ('00555 ', 'SIGNIFICANCE'): 0.06899934542156537,
 ('0

In [145]:
def vms(tfidf, query):
    query_tokens = [token.upper() for token in word_tokenize(query) if token not in nltk.corpus.stopwords.words('english')]
    query_weights = {}
    docs_weights = {}
    D = {}
    dot_QD = {}
    q_d_vec_diffs = {}

    tf_query = Counter(query_tokens)

    # compute each token weight with respect to the QUERY
    for token in query_tokens:
        tf = tf_query[token] / len(query_tokens)
        if token in li.keys():
            idf = N / len(set(li[token]))
        else:
            idf = 0
        query_weights[token] = tf * idf

    # compute each token weight with respect to EACH DOCUMENT
    for (doc_id, token), weight in tfidf.items():
        if token in query_tokens:
            if doc_id not in docs_weights:
                docs_weights[doc_id] = []
            docs_weights[doc_id].append((token, weight))
            
    for doc_id, token_weight_list in docs_weights.items():
    # sorting docs_weight
        docs_weights[doc_id] = sorted(token_weight_list)

    for doc_id, token_weight_list in docs_weights.items():
        common_QD_tokens = sorted(token for (token, _) in token_weight_list if token in query_weights.keys())
        qw_vector = np.array([query_weights[token] for token in common_QD_tokens])

        # create document vector containing weights for common tokens
        doc_vector = np.array([weight for token, weight in token_weight_list if token in common_QD_tokens])
        
        
        sum_of_squares = sum(dw**2 for _, dw in token_weight_list)
        D[doc_id] = math.sqrt(sum_of_squares)

        # compute scalar products of Query weights vector with each document weights vector
        dot_QD[doc_id] = np.dot(qw_vector, doc_vector)
        q_d_vec_diffs[doc_id] = qw_vector - doc_vector
    Q = math.sqrt(sum([qw**2 for qw in query_weights.values()]))
    return dot_QD, Q, D, q_d_vec_diffs

In [146]:
dot_query = {}
Q_query = {}
D_query = {}
vec_diffs_query = {}

for id, query in queries.items():
    (dot_query[id], Q_query[id], D_query[id], vec_diffs_query[id]) = vms(modelo_vetorial, query)

In [147]:
def cosine_similarity(dot_products, qw_vector_mag, dw_vector_mag):
    cs_docs = {}

    for (id, dps) in dot_products.items():
        for doc_id, dp in dps.items():
            if id not in cs_docs:
                cs_docs[id] = []
            
            cs_docs[id].append((doc_id, dps[doc_id] / (qw_vector_mag[id] * (dw_vector_mag[id])[doc_id])))
            # print(dps[doc_id])
            # print(qw_vector_mag[id])
            # print((dw_vector_mag[id])[doc_id])
    return cs_docs
    

In [148]:
similarities = cosine_similarity(dot_query, Q_query, D_query)
similarities

{1: [('00001 ', 0.9957627827097757),
  ('00006 ', 0.9688006284839182),
  ('00016 ', 0.6616875123472102),
  ('00018 ', 0.6549123333069956),
  ('00048 ', 0.6549123333069956),
  ('00057 ', 0.6629936937548412),
  ('00058 ', 0.6589910583130738),
  ('00061 ', 0.6616875123472101),
  ('00062 ', 0.6559377121248586),
  ('00079 ', 0.6549123333069955),
  ('00081 ', 0.6549123333069955),
  ('00104 ', 0.6616875123472102),
  ('00111 ', 0.6574118993609054),
  ('00123 ', 0.6616875123472102),
  ('00133 ', 0.9892634034492646),
  ('00146 ', 0.6629936937548413),
  ('00159 ', 0.9983811645951437),
  ('00160 ', 0.9768939821103736),
  ('00176', 0.9983811645951434),
  ('00179', 0.9983811645951437),
  ('00187', 0.9983811645951436),
  ('00188', 0.6596410153442611),
  ('00200', 0.6577847678270783),
  ('00202', 0.6549123333069956),
  ('00212', 0.9983811645951436),
  ('00213', 0.6616875123472102),
  ('00238', 0.6591836065216453),
  ('00247', 0.6549123333069955),
  ('00253', 0.6629936937548413),
  ('00262', 0.66168751

In [149]:
vec_diffs_query

{1: {'00001 ': array([0.23125743, 0.2306329 , 1.98969135, 2.28038389]),
  '00006 ': array([0.23158207, 0.23095153, 2.00462324, 2.31744656]),
  '00016 ': array([0.22854419, 0.22796982, 1.99829805]),
  '00018 ': array([2.04900981]),
  '00048 ': array([2.01686457]),
  '00057 ': array([0.22020564, 0.21978546, 1.96772633]),
  '00058 ': array([0.23135666, 0.23073029, 2.01783701]),
  '00061 ': array([0.23152294, 0.2308935 , 2.04396785]),
  '00062 ': array([0.23149777, 0.23086879, 1.87223749]),
  '00079 ': array([2.03211876]),
  '00081 ': array([2.03345178]),
  '00104 ': array([0.2319548 , 0.23131737, 2.05058909]),
  '00111 ': array([0.23012621, 0.22952259, 1.91643657]),
  '00123 ': array([0.23114452, 0.23052207, 2.03816588]),
  '00133 ': array([0.22274346, 0.22227635, 2.02609053, 2.31886501]),
  '00146 ': array([0.22680397, 0.22626178, 2.01830864]),
  '00159 ': array([0.23062206, 0.23000927, 2.0301557 , 2.32316269]),
  '00160 ': array([0.22820953, 0.22764135, 1.9572507 , 2.17014619]),
  '0017

In [150]:
def get_results(cs):
    ranked_docs = {}
    winning_docs = {}

    for id, docs_cs in cs.items():
        ranked_docs[id] = list(sorted(docs_cs, key=lambda x: x[1], reverse=True))

    for id, docs_cs in cs.items():
        winning_docs[id] = ranked_docs[id][0]
    return winning_docs

    

In [151]:
results = get_results(similarities)

In [152]:
with open(results_path, "w", newline="") as file:
    writer = csv.writer(file, delimiter=";")
    writer.writerow(["query id", "[first place doc, cosine similarity to query]"]) 
    for q_id, (d_id, cs) in results.items():
        row = [q_id, f"[{d_id},{cs}]"]
        writer.writerow(row)